<a href="https://colab.research.google.com/github/ahzaidy/Programs/blob/main/CPSC_5440_HW21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, TensorDataset
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pickle
from google.colab import drive

# Mount Google Drive
drive.mount("/content/drive")

# Load CIFAR-100 dataset from Google Drive
with open('/content/drive/My Drive/train', 'rb') as file:
    train_dict = pickle.load(file, encoding='bytes')

with open('/content/drive/My Drive/test', 'rb') as file:
    test_dict = pickle.load(file, encoding='bytes')

# Extract data and labels
train_data = torch.tensor(train_dict[b'data'], dtype=torch.float32).reshape(-1, 3, 32, 32) / 255.0
train_labels = torch.tensor(train_dict[b'fine_labels'], dtype=torch.long)
test_data = torch.tensor(test_dict[b'data'], dtype=torch.float32).reshape(-1, 3, 32, 32) / 255.0
test_labels = torch.tensor(test_dict[b'fine_labels'], dtype=torch.long)

# Create datasets
train_dataset = TensorDataset(train_data, train_labels)
test_dataset = TensorDataset(test_data, test_labels)

class CIFAR100Net(nn.Module):
    def __init__(self, units, hidden_activation, output_activation):
        super(CIFAR100Net, self).__init__()
        self.hidden_activation = getattr(nn, hidden_activation)()
        self.output_activation = getattr(nn, output_activation)()
        self.fc1 = nn.Linear(3 * 32 * 32, units)
        self.hidden_layers = nn.ModuleList([nn.Linear(units, units) for _ in range(4)])
        self.output_layer = nn.Linear(units, 100)

    def forward(self, x):
        x = x.view(-1, 3 * 32 * 32)
        x = self.hidden_activation(self.fc1(x))
        for layer in self.hidden_layers:
            x = self.hidden_activation(layer(x))
        x = self.output_activation(self.output_layer(x))
        return x

def train_cifar100(config, train_dataset=None, test_dataset=None, checkpoint_dir=None):
    transform = transforms.Compose([transforms.Normalize((0.5,), (0.5,))])

    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_subset, val_subset = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_subset, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=config['batch_size'], shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

    device = torch.device("cpu")
    model = CIFAR100Net(config['units'], config['hidden_activation'], config['output_activation']).to(device)

    criterion = getattr(nn, config['loss'])()
    optimizer = getattr(optim, config['optimizer'])(model.parameters(), lr=config['lr'])

    for epoch in range(10):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        val_loss = 0.0
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = correct / total
        tune.report(loss=val_loss, accuracy=val_accuracy)

config = {
    'units': tune.choice([120, 240]),
    'hidden_activation': tune.choice(['ReLU', 'Sigmoid']),
    'output_activation': tune.choice(['Softmax', 'Sigmoid']),
    'loss': tune.choice(['CrossEntropyLoss', 'MSELoss']),
    'optimizer': tune.choice(['Adam', 'Adagrad']),
    'batch_size': tune.choice([1000, 2000]),
    'lr': tune.loguniform(1e-4, 1e-1)
}

ray.init(ignore_reinit_error=True)
scheduler = ASHAScheduler(
    metric='accuracy',
    mode='max',
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

analysis = tune.run(
    tune.with_parameters(train_cifar100, train_dataset=train_dataset, test_dataset=test_dataset),
    config=config,
    num_samples=10,
    scheduler=scheduler,
    resources_per_trial={'cpu': 2}
)

df = analysis.results_df
best_trial = analysis.get_best_trial('accuracy', 'max', 'last')
best_config = best_trial.config
best_accuracy = best_trial.last_result['accuracy']

print(f'Best Hyperparameters: {best_config}')
print(f'Best Accuracy: {best_accuracy:.4f}')

plt.figure(figsize=(10, 5))
sns.lineplot(data=df, x='training_iteration', y='accuracy', hue='hidden_activation')
plt.xlabel('Training Iteration')
plt.ylabel('Validation Accuracy')
plt.title('Hyperparameter Tuning Results')
plt.grid()
plt.savefig("/content/drive/My Drive/hyperparameter_tuning_plot.png", dpi=300)
plt.show()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2025-03-07 05:49:25,157	INFO worker.py:1672 -- Calling ray.init() again after it has already been called.


+-----------------------------------------------------------------------+
| Configuration for experiment     train_cifar100_2025-03-07_05-49-27   |
+-----------------------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator                |
| Scheduler                        AsyncHyperBandScheduler              |
| Number of trials                 10                                   |
+-----------------------------------------------------------------------+

View detailed results here: /root/ray_results/train_cifar100_2025-03-07_05-49-27
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-03-07_05-45-45_878621_2122/artifacts/2025-03-07_05-49-27/train_cifar100_2025-03-07_05-49-27/driver_artifacts`

Trial status: 10 PENDING
Current time: 2025-03-07 05:49:28. Total running time: 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------------

2025-03-07 05:50:28,307	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 2.0 CPUs and 1.0 GPUs per trial, but the cluster only has 2.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


Trial status: 10 PENDING
Current time: 2025-03-07 05:50:28. Total running time: 1min 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                   status       units   hidden_activation     output_activation     loss               optimizer       batch_size            lr |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
| train_cifar100_ee81f_00000   PENDING        120   Sigmoid               Softmax               MSELoss            Adam                  1000   0.00433892  |
| train_cifar100_ee81f_00001   PENDING        120   Sigmoid               Sigmoid               MSELoss            Adam                  1000   0.000139288 |
| train_cifar100_ee81f_00002   PENDING        120   ReLU                  Sigmo

2025-03-07 05:51:28,360	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 2.0 CPUs and 1.0 GPUs per trial, but the cluster only has 2.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


Trial status: 10 PENDING
Current time: 2025-03-07 05:51:28. Total running time: 2min 0s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                   status       units   hidden_activation     output_activation     loss               optimizer       batch_size            lr |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
| train_cifar100_ee81f_00000   PENDING        120   Sigmoid               Softmax               MSELoss            Adam                  1000   0.00433892  |
| train_cifar100_ee81f_00001   PENDING        120   Sigmoid               Sigmoid               MSELoss            Adam                  1000   0.000139288 |
| train_cifar100_ee81f_00002   PENDING        120   ReLU                  Sigmo

2025-03-07 05:51:29,371	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2025-03-07 05:51:29,376	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_cifar100_2025-03-07_05-49-27' in 0.0039s.
2025-03-07 05:51:29,389	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2025-03-07 05:51:29,400	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 10 trial(s):
- train_cifar100_ee81f_00000: FileNotFoundError('Could not fetch metrics for train_cifar100_ee81f_00000: both result.json and progress.csv were not found at /root/ray_results/train_cifar100_2025-03-07_05-49-27/train_cifar100_ee81f_00000_0_batch_size=1000,hidden_activation=Sigmoid,loss=MSELoss,lr=0.0043,optimize

Trial status: 10 PENDING
Current time: 2025-03-07 05:51:29. Total running time: 2min 1s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
| Trial name                   status       units   hidden_activation     output_activation     loss               optimizer       batch_size            lr |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------+
| train_cifar100_ee81f_00000   PENDING        120   Sigmoid               Softmax               MSELoss            Adam                  1000   0.00433892  |
| train_cifar100_ee81f_00001   PENDING        120   Sigmoid               Sigmoid               MSELoss            Adam                  1000   0.000139288 |
| train_cifar100_ee81f_00002   PENDING        120   ReLU                  Sigmo

AttributeError: 'NoneType' object has no attribute 'config'

In [ ]:
!pip install ray[tune]

In [ ]:
!pip install ray